# Đồ án Thực Hành Lập Trình Khoa Học Dữ Liệu

In [ ]:
import sys
sys.executable

## Import thư viện

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import re
from collections import Counter

## Thu thập dữ liệu

- Nhóm sử dụng dữ liệu "steam_games.csv" được thu thập sẵn ở trên kaggle. Dữ liệu này là về các game hiện có ở trên steam. Nguồn của dữ liệu được lấy tại: https://www.kaggle.com/datasets/tristan581/all-55000-games-on-steam-november-2022
- LICENSE của dữ liệu là [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/), người dùng được phép sử dụng với mục đích phi thương mại và phải credit cho chủ sở hữu
- Người ta đã thu thập dữ liệu này bằng cách sử dụng API của steam store và API của web thứ ba (steamspy.com)

## Khám phá dữ liệu

### Đọc dữ liệu vào dataframe

In [ ]:
steam_df = pd.read_csv('steam_games.csv', sep = ';')
steam_df.head()

### Dữ liệu có bao nhiêu dòng, bao nhiêu cột?

Tính số dòng và số cột lưu vào hai biến num_rows và num_cols

In [ ]:
num_rows, num_cols = steam_df.shape
num_rows, num_cols

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng cho biết thông tin của từng sản phẩm hiện có trong steam, không có dòng nào có ý nghĩa khác với các dòng còn lại.

### Tiến hành kiểm tra các dòng có bị lặp không

Kiểm tra bằng cách lưu kết quả vào biến `have_duplicated_rows`. Biến này có giá trị True nếu dữ liệu có các dòng bị lặp và False nếu ngược lại.

In [ ]:
duplicated_list = steam_df.duplicated()
if True in duplicated_list:
    have_duplicated_rows = True
else:
    have_duplicated_rows = False
have_duplicated_rows

Không có dòng nào bị lặp cả

### Mỗi cột có ý nghĩa gì?

Theo mô tả trên kaggle, các cột có ý nghĩa như sau:
- App ID: ID của sản phẩm
- Name: Tên sản phẩm
- Short Description: Phần giới thiệu sản phẩm
- Developer: Nhà phát triển sản phẩm
- Publisher: Nhà phân phối sản phẩm
- Genre: Các thể loại của sản phẩm
- Tags: Các tag được gán cho sản phẩm (từ người dùng)
- Type: Cho biết sản phẩm thuộc loại game hay là phần cứng (các phần mềm không phải game cũng được gán giá trị là game)
- Categories: Các hạng mục/đặc trưng của sản phẩm
- Owners: Số lượng người sở hữu sản phẩm
- Positive Reviews: Số lượt đánh giá tích cực 
- Negative Reviews: Số lượt đánh giá tiêu cực
- Price: Giá bán hiện tại của game (tính theo USD)
- Initial Price: Giá bán của game thời điểm mới phát hành (USD)
- Discount: Phần trăm giảm giá của game
- CCU: Số lượng người chơi đồng thời cao nhất (tính vào ngày 8/11/2021)
- Languages: Các ngôn ngữ mà sản phẩm có hỗ trợ
- Platforms: Các nền tảng mà sản phẩm hỗ trợ
- Release Date: Ngày phát hành sản phẩm
- Required Age: Độ tuổi giới hạn
- Website: Trang web của nhà phát triển/nhà phân phối
- Header Image: Đường link lưu ảnh của sản phẩm

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [ ]:
steam_df.columns

- Lúc đọc dữ liệu từ "steam_games.csv", ta có thể thấy warning là:"Columns (19) have mixed types.", nghĩa là cột 19(Required Age) có dtype không đồng nhất.<br>
- Release Date cần đưa về dạng datetime<br>

In [ ]:
steam_df.head()

In [ ]:
col_dtypes=steam_df.dtypes
col_dtypes

In [ ]:
steam_df['Required Age'].unique()

In [ ]:
def open_object_dtype(s):
    dtypes = set(s.apply(type))
    return dtypes
open_object_dtype(steam_df['Required Age'])
# USK 0 =PEGI 3: phuf hopwj

- Steam(*https://store.steampowered.com/*) là một nền tảng phân phối trực tuyến, quản lý bản quyền kỹ thuật số, trò chơi điện tử nhiều người chơi, và dịch vụ giao tiếp xã hội trên nền internet phát triển bởi Valve Corporation. Nên các trò chơi điện tử của các quốc gia khác nhau có giới hạn độ tuổi(**Required Age**) khác nhau, sau đây để dễ so sánh và tổng hợp, nhóm mình(em) xin được quy những giới hạn độ tuổi về thang PEGI(*https://pegi.info/what-do-the-labels-mean*).<br>
- Ở cột **Requied Age** ta xử lý chuyển đổi thông tin thành dạng sau đây:<br>
	- 3(PEGI 3) – suitable for all ages, 
	- 7(PEGI 7) – suitable for young children (age>=7)
	- 12(PEGI 12) – suitable for children 12 and over (age>=12)
	- 16(PEGI 16) – suitable for children 16 and over (age>=16)
	- 18(PEGI 18) – Only suitable for adults (age>=18)

In [ ]:
def trans_age(type_age):
    l_age=re.findall(r'\d{1,2}',str(type_age))
    age=int(l_age[0])
    if age>=18:
        return 18
    elif age>=16:
        return 16
    elif age>=12:
        return 12
    elif age>=7:
        return 7
    return 3
steam_df['Required Age']=steam_df['Required Age'].apply(trans_age)

In [ ]:
# steam_df['Release Date']=pd.to_datetime(steam_df['Release Date'],format='%Y/%m/%d')
steam_df['Release Date']=steam_df['Release Date'].astype('datetime64[s]')

### Với mỗi cột, các giá trị được phân bố như thế nào?

In [ ]:
#YOUR CODE HERE

numeric_cols=steam_df[steam_df.select_dtypes(include=[int,'datetime64[ns]']).columns]

def missing_ratio(col):
    return np.round(col.isnull().sum()/len(col)*100,4)
def min(col):
    return col.min()
def max(col):
    return col.max()

num_col_profiles_df=numeric_cols.agg([missing_ratio,min,max])
num_col_profiles_df



- Ta thực hiện kiểm tra giá trị phân bố của các biến object:
	- Với các cột **Name, Short Description,Website, Header Image** là những cột chứa thông tin đặc trưng của từng game, nên giá trị phân bố là trải rộng đều với nhau
	- Với cột **Tags** chứa thông tin của từng Genre của game nên ở đây ta cũng không xét phân bố như thế nào

In [ ]:
steam_df.head()

In [ ]:
# object_cols=steam_df[]
# display(object_cols)
object_cols=['Developer', 'Publisher', 'Genre', 'Type', 'Categories', 'Owners', 'Languages', 'Platforms']
# steam_df.select_dtypes(include=[object]).columns
print(object_cols)
# def str_to_list(col):
#     for i in col.index:
#         if type(col.at[i])==str:
#             col.at[i]=col.at[i].split(',')
#     return col
def split_str(st):
    if not (st is np.nan):
        return st.split(',')
    return []

<h3 style="color: orange">Developer</h3>

In [ ]:
pd.set_option("display.max_rows",None)
def lower_str(st):
    if not (st is np.nan):
        return st.lower()
    return ""
def convert_co_inc(st):
    if not (st is np.nan):
        st = re.sub(" *ltd.?[ .]*","ltd.,", st)
        st= re.sub(" *inc.?[ .]*","inc.,", st)
        st= re.sub(" *llc.?[ .]*","llc.,", st)
        return re.sub(r" *, *(?!inc|ltd|llc)",";", st)
    return ""
def split_dev(st):
    if not (st is np.nan):
        return st.split(';')
    return []

steam_df.Developer.apply(lower_str).apply(convert_co_inc).apply(split_dev).explode().replace("",np.nan).value_counts()

<h3 style="color: orange">Publisher</h3>

In [ ]:
steam_df.Publisher.apply(lower_str).apply(convert_co_inc).apply(split_dev).explode().replace("",np.nan).value_counts()

<h3 style="color: orange">Genre</h3>

In [ ]:
col=steam_df.Genre
col=col.apply(split_str)
col=col.explode()
col=col.str.strip()
ratio_value=col.value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

<h3 style="color: orange">Type</h3>

In [ ]:
ratio_value=steam_df.Type.value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

<h3 style="color: orange">Categories</h3>

In [ ]:
col=steam_df.Categories
col=col.apply(split_str)
col=col.explode()
col=col.str.strip()
ratio_value=col.value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

<h3 style="color: orange">Owners</h3>

In [ ]:
def split_owner(st):
    if not(st is np.nan):
        st=st.replace(',','')
        value=st.split('..')
        return (int(value[0]),int(value[1]))
    return np.nan
ratio_value=steam_df.Owners.apply(split_owner).value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

<h3 style="color: orange">Languages</h3>

In [ ]:
def clean_lang(lang_str):
    if not (lang_str is np.nan):
        return lang_str.replace('\r\n',',').replace('[b]*[/b]','').replace(';','').replace('#lang_slovakian', 'Slovak')
    return ""

col=steam_df.Languages
# Xử lý dữ liệu
col=col.apply(clean_lang).apply(split_str)
col=col.explode()
col=col.str.strip()
col=col.apply(lambda st: st.split('-')[0].strip())
col=col.replace('',np.nan)

# Xem các giá trị trong cột được phân bố như thế nào
ratio_value=col.value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

<h3 style="color: orange">Platforms</h3>

In [ ]:
col = steam_df['Platforms']
col=col.apply(split_str)
col=col.explode()
col=col.str.strip()
ratio_value=col.value_counts(normalize=True).multiply(100)
num_value=len(ratio_value)
print(num_value)
print(ratio_value)

## Đưa ra các câu hỏi

### Câu 1: Các thể loại có lượt người tag nhiều nhất. Tương quan giữa lượt tag thể loại của game đó với thể loại chính thức của game?

**Ý nghĩa khi trả lời câu hỏi:** 
* Có được cái nhìn tổng quan về từng thể loại, thể loại được người chơi công nhận nhiều nhất (thể loại mỗi game do chính người chơi tự nhìn nhận và định nghĩa)
* Biết được liệu rằng thể loại chính thức của game có giống với những gì mà người dùng trải nghiệm hay không?

**Các bước cần làm để để trả lời câu hỏi:**

* Đầu tiên ta cần đưa thông tin về lượt tag trong từng game về dưới dạng dictionary với key là thể loại và value là số lượng người dùng tag game với thể loại đó
* Tiếp đến để xét tổng quan lượt tag của từng thể loại, ta cần cộng các giá trị lượt tag của từng thể loại xuyên suốt tất cả các game với nhau và lưu vào một Series với index là tên thể loại và value là tổng số lượt tag của thể loại đó
* Với ý thứ hai, ta sẽ xét các thể loại (genres) chính thức của game bằng cách chấm điểm dựa vào số lượt tag của từng thể loại, cụ thể như sau:
    * Đối với các thể loại chính thức có ở trong phần tag, ta tính điểm bằng tỉ lệ giữa lượt tag của thể loại đó với tổng lượt tag
    * Đối với các thể loại chính thức nhưng không có trong phần tag thì ta cho điểm bằng 0
    * Từ điểm của từng thể loại, ta tính trung bình điểm bằng cách lấy tổng điểm của các thể loại chính thức đã tính ở trên chia cho tổng số thể loại.

## Tiền xử lý và phân tích dữ liệu để trả lời cho từng câu hỏi

### Tiền xử lý và phân tích cho câu 1:

**Bước 1:** Ta tạo một dataframe mới có 2 cột là `Genre` và `Tags` lấy từ dataframe gốc, với cột `Tags`, ta tiến hành chuyển đổi nó về dạng dictionary với key là thể loại và values là số lượng người dùng tag game với thể loại đó.

In [ ]:
def split_genre(x):
    if x is np.nan:
        return np.nan
    return [i.strip() for i in x.split(',')]
def split_tags(x):
    if x is np.nan:
        return []
    return [i.strip() for i in x.split(',')]
genres_tag_df = steam_df[["Genre", "Tags"]].copy()
genres_tag_df["Genre"] = genres_tag_df["Genre"].apply(split_genre)
genres_tag_df["Tags"] = genres_tag_df["Tags"].apply(split_tags)
def convert_to_dict(x: list):
    res = {}
    if len(x) == 0:
        return np.nan
    for string in x:
        key, val = string.split(': ')
        res[key] = int(val)
    return res
genres_tag_df["Tags"] = genres_tag_df["Tags"].apply(convert_to_dict)
genres_tag_df

Để tiện cho việc xử lý các bước sau, ta tiến hành bỏ đi các dòng có giá trị NaN.

In [ ]:
print('Before:', genres_tag_df.shape[0])
genres_tag_df = genres_tag_df.dropna()
print('After:', genres_tag_df.shape[0])

Có thể thấy số lượng các dòng bị bỏ đi là không đáng kể nên nhìn chung việc này sẽ không ảnh hưởng mấy đến kết quả cuối cùng.

**Bước 2:** Sau khi đã chuyển dữ liệu của cột Tags thành dictionary rồi thì ta tiến hành tính tổng số lượt tag của từng thể loại, lưu kết quả vào Series `tolal_num_tags` với mỗi dòng biểu thị số lượng tổng mỗi tag.

In [ ]:
c = Counter()
def sum_tags(row):
    c.update(row)
    return row
genres_tag_df["Tags"].apply(sum_tags)
total_num_tags = pd.Series(c, index = c.keys())
total_num_tags = total_num_tags.sort_values(ascending = False)
print(total_num_tags)
print(len(total_num_tags))

Ta có thể thấy được top 5 thể loại đang được người chơi để ý và công nhận nhất là `Action`, `Adventure`, `Indie`, `Singleplayer`, `Casual`

**Bước 3:** Chấm điểm cho các game dựa vào độ tương đồng giữa thể loại chính thức và thể loại do người chơi tag. Đầu tiên ta tiến hành tính điểm như đã mô tả ở trên và lưu dữ liệu vào một Series có tên `score`.

In [ ]:
def scoring(row):
    res = 0
    genres = row["Genre"]
    tags = row["Tags"]
    for genre in genres:
        if genre in tags.keys():
            score = tags[genre] / sum(tags.values())
        else:
            score = 0
        res += score
    return res/len(genres)
score = genres_tag_df.apply(scoring, axis = 1)
print(score)
print('Mean:', score.mean())

Theo như ta thấy, điểm trung bình độ tương đồng có vẻ thấp. Một cách để giải thích cho việc này chính là các thể loại do chính người dùng tự định nghĩa và tag là rất đa dạng, ta có thể nhìn thấy, số lượng thể loại được tag lên đến hơn 400 thể loại khác nhau trong khi số lượng thể loại chính thức do chính nhà phát hành đưa ra chỉ đâu đó tầm 30 thể loại. 
Bây giờ ta cùng thử lọc ra các game có độ tương đồng lớn hơn 0.5 xem sao.

In [ ]:
gt05 = pd.Series(score.loc[score>=0.5])
print(gt05)
print(len(gt05)/len(score))

Bây giờ ta cùng thử xem liệu các game có số lượng trên 0.5 có liên quan gì tới số lượng người sở hữu không

Ta tạo một dataframe có hai cột là `Score` và `Owners` lần lượt là số điểm tương đồng và số lượng người sở hữu của game đó.

In [ ]:
owners = steam_df["Owners"].apply(split_owner)
score_owner_df = gt05.to_frame(name = "Score").join(owners)
score_owner_df

Tiếp đến, ta sẽ đếm số lượng đạt game 0.5 theo từng khoảng lượng người dùng.

In [ ]:
count_score_owner = score_owner_df.groupby(["Owners"])["Owners"].count()
count_score_owner

Thử trực quan hóa kết quả bằng biểu đồ bar

In [ ]:
ax = count_score_owner.plot.barh()
ax.set_xlabel("Nums of games with high score")
ax.set_title("Nums of high score games according to owners range")

Qua đây có thể thấy được là game càng có nhiều người chơi thì điểm tương đồng càng ít, điều này là vì càng nhiều người chơi thì sẽ có càng nhiều tag và độ đa dạng của tag sẽ càng nhiều, từ đó khiến tới việc điểm tương đồng bị giảm